In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import SimpleITK as sitk
from tqdm import tqdm
from pathlib import Path


import utils
import shutil

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Register atlases to all test images

In [28]:
# Define all necessary paths
base_path = Path('../').resolve().parent
test_set_path = base_path / 'data' / 'test-set'
params_path =  base_path / 'elastix' / 'parameter_maps'
test_imgs_dir =   test_set_path / 'testing-images'
test_labels_dir = test_set_path / 'testing-labels'
test_masks_dir =  test_set_path / 'testing-mask'
output_path =     test_set_path

our_atlas_path = base_path / 'data' / 'atlases' / 'Parameters.Par0009.affine'
mni_atlas_path = base_path / 'data' / 'MNITemplateAtlas'

# Define parameter maps to use
param_file_path =  base_path / 'elastix' / 'parameter_maps' / 'Par0010affine.txt'

In [29]:
# Separating Labels for MNI atlas
mni_atlas_template = sitk.ReadImage(str(mni_atlas_path / 'template.nii.gz'))
mni_atlas_labels = sitk.ReadImage(str(mni_atlas_path / 'atlas.nii.gz')) # probability maps image 
mni_atlas_labels_array = sitk.GetArrayFromImage(mni_atlas_labels) # probability maps array

atlas_background = mni_atlas_labels_array[0, :, :, :]
atlas_csf = mni_atlas_labels_array[1, :, :, :]
atlas_gm = mni_atlas_labels_array[2, :, :, :]
atlas_wm = mni_atlas_labels_array[3, :, :, :]
 
utils.save_segementations(atlas_background, mni_atlas_template , str(mni_atlas_path/'p_atlas_background.nii.gz'))
utils.save_segementations(atlas_csf, mni_atlas_template , str(mni_atlas_path/'p_atlas_csf.nii.gz'))
utils.save_segementations(atlas_gm, mni_atlas_template , str(mni_atlas_path/'p_atlas_gm.nii.gz'))
utils.save_segementations(atlas_wm, mni_atlas_template , str(mni_atlas_path/'p_atlas_wm.nii.gz'))

In [33]:
# Registration of the atlases to each test image

# Atlas paths
our_atlas_template_path = our_atlas_path / 'mean_volume.nii.gz'
mni_atlas_template_path = mni_atlas_path / 'template.nii.gz'

atlas_path = {'our_atlas': our_atlas_path, 'mni_atlas': mni_atlas_path}
atlas_template_path = {'our_atlas': our_atlas_template_path, 'mni_atlas': mni_atlas_template_path}
atlas_map_names = ['p_atlas_background', 'p_atlas_csf', 'p_atlas_gm', 'p_atlas_wm']

# Read and modify parameters file
field_value_pairs = [('ResultImageFormat', 'nii.gz'), ('WriteResultImage', 'true')]
utils.modify_field_parameter_map(field_value_pairs, param_file_path)


for j, fixed_img_path in enumerate(test_imgs_dir.iterdir()):
    
    print(fixed_img_path)
    fix_name = fixed_img_path.name.rstrip('.nii.gz')
    
    mask_path = test_masks_dir / f'{fix_name}_1C.nii.gz'
    
    # For each atlas 
    for atlas_name in ['our_atlas', 'mni_atlas']:
        result_path = output_path / atlas_name / fix_name
        result_path.mkdir(exist_ok=True, parents=True)

        res_img_path = result_path / f'{atlas_name}_template.nii.gz'
        res_mask_path = result_path / f'{atlas_name}_1C.nii.gz'

        # Register
        transform_map_path = utils.elastix_wrapper(fixed_img_path, atlas_template_path[atlas_name], res_img_path, param_file_path)

        # Correct transformation parameters file
        field_value_pairs = [('ResultImageFormat', 'nii.gz'), ('ResultImagePixelType', "float"), ('FinalBSplineInterpolationOrder', '2.0')]
        utils.modify_field_parameter_map(field_value_pairs, transform_map_path)
        
        # Transform brain_mask
        utils.transformix_wrapper(mask_path, res_mask_path, transform_map_path)  
    
        for label_name in atlas_map_names:
            res_lab_path = result_path / f'{atlas_name}_{label_name}.nii.gz'
            lab_path = str(atlas_path[atlas_name] / label_name)
            
            # Transform labels
            utils.transformix_wrapper(lab_path, res_lab_path, transform_map_path)

C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1003.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1004.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1005.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1018.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1019.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1023.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1024.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1025.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1038.nii.gz
C:\Users\kaout\OneDrive\Documents\GitHub\joaco\misa_lab\data\test-set\testing-images\1039.nii.gz
C:\Users\kaout\OneDrive\Docume